### Set Up

In [1]:
import numpy as np
from itertools import product

### Helper functions to compute the kernel of a matrix over $\mathbb{F}_2$

The following function row reduces a matrix $M$ over $\mathbb{F}_2$ to row-echelon form returns a list of coordinates corresponding to the leading columns of the row reduced matrix. 

In [2]:
def row_reduce(M):
    # Keep track of which rows are complete
    curr = 0
    # Keep track of leading columns to complete row echelon form
    leading_pos = []
    
    k, n = M.shape

    for c in range(n):
        # Search for the first 1 in the column
        found = False
        for r in range(curr, k):
            if M[r][c] == 1:
                # Bring row to the current level
                if not found:
                    M[[r, curr]] = M[[curr, r]]
                    leading_pos.insert(0, (curr, c))
                    found = True
                # Use the found 1 to eliminate 1s below
                else:
                    M[r] =  (M[r] + M[curr]) % 2
        
        # If we found a 1 in the row, move on to the next row
        if found:
            curr += 1             
    
    # Reduce back up the columns to complete row echelon form
    for r, c in leading_pos:
        for i in range(r):
            if M[i][c] == 1:
                M[i] = (M[i] + M[r]) % 2

    return leading_pos

The following function implements an efficient algorithm to find the kernel of of a matrix $M$ over $\mathbb{F}_2$. This function returns the (finite) set of all vectors in the kernel.

Given a $k\times n$ matrix $M$ of rank $k$, express it in row-echelon form, interchange columns and rows as needed to express the matrix in the form $[I_{k\times k} | Q]$ where $Q$ is a $k\times(n-k)$ matrix. Form the matrix $K = [Q^T | I_{(n-k)\times (n-k)}]$. Now undo the column permutations to get the basis vectors for kernel of $M$. 

In [3]:
def kernel(M):
    # Begin with the row reduction
    leading_pos = row_reduce(M)
    
    # Drop all zero rows
    M = np.array([row for row in M if not all(row == 0)])
    
    k, n = M.shape

    # Column order
    cols = np.array(range(n))

    M = M.T
    # Move leading column to row position
    for r, c in reversed(leading_pos):
        M[[c, r]] = M[[r, c]]
        cols[c], cols[r] = cols[r], cols[c]

    Q = M[k:]
    K = np.concatenate((Q.T, np.eye(n-k, dtype = int)))

    # Reshuffle the columns to original order
    # B is a basis for kernel
    B = K[cols.argsort()].T

    # Entire kernel
    S = [list((B.T @ x) % 2) for x in product([0,1], repeat=n-k)]
    return S

Set seed for reproducibility. 

In [4]:
np.random.seed(2)

### Bob receives the matrix $P$ from Alice.

In [5]:
P = np.array([[0, 1, 1, 0, 0],
              [0, 1, 0, 0, 1],
              [1, 0, 1, 1, 1],
              [1, 1, 0, 0, 1],
              [0, 0, 0, 1, 0],
              [1, 0, 0, 1, 0],
              [0, 0, 1, 1, 1],
              [1, 1, 0, 1, 1],
              [1, 0, 0, 0, 0],
              [0, 1, 1, 1, 0],
              [1, 1, 0, 1, 0],
              [0, 0, 0, 0, 1],
              [0, 0, 1, 0, 0],
              [0, 1, 0, 1, 1]])

n = P.shape[1]

### Bob runs Kahanamoku-Meyer's algorithm to reveal the secret codeword s

Pick $\textbf{d} \in \mathbb{F}_2^n$ non-zero uniformly at random.

In [6]:
d = np.random.randint(0, 2, n)
print(d)

[0 1 1 0 0]


Generate a large number (say $3n$) of vectors $\textbf{m}_i$, forming the rows of a matrix $M$. For each:

(i) Pick $\textbf{e} \in \mathbb{F}_2^n$ non-zero uniformly at random.

(ii) Let $\textbf{m}_i = \sum_{\substack{\textbf{p}\in P\\ \textbf{p}\cdot\textbf{d} = \textbf{p}\cdot\textbf{e} = 1}} \textbf{p}$

In [7]:
M = []
for _ in range(3*n):
    e = np.random.randint(0, 2, n)
    
    rows_to_sum = [p for p in P if e.dot(p)%2 == 1 and d.dot(p)%2 == 1]
    if not rows_to_sum:
        continue
    
    m_i = sum(rows_to_sum)
    M.append(m_i % 2)

M = np.array(M)
print(M)

[[0 1 0 1 1]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [1 1 1 1 0]
 [0 1 0 1 0]
 [1 0 1 0 1]
 [1 0 1 0 0]
 [0 1 0 1 0]
 [1 0 1 0 1]
 [0 1 0 1 1]
 [1 1 0 1 0]
 [1 0 0 0 1]
 [1 1 1 1 1]
 [0 0 0 0 1]]


Find the kernel of $M$.

In [8]:
S = kernel(M)
print(S)

[[0, 0, 0, 0, 0], [0, 1, 0, 1, 0]]


For each candidate vector $\textbf{s}_i$:

(i) Extract $P_{\textbf{s}_i}$ from $P$ be deleting all rows of $P$ orthogonal to $\textbf{s}_i$.

(ii) Check if $P_{\textbf{s}_i}$ generates a quadratic residue code by checking codeword weights.



In [9]:
for s in S:
    # Submatrix potentially generating a quadratic residue code
    P_s = np.array([p for p in P if p.dot(s) % 2 == 1])
    
    if P_s.size == 0:
        continue

    # Compute explicitly the code generated by the matrix P_s
    C = {tuple((P_s @ x) % 2) for x in product([0,1], repeat=n)}
    
    # Check codeword weights
    if all(sum(c) % 4 in [0,3] for c in C):
        print(s)
        break

[0, 1, 0, 1, 0]


Pow! We have found Alice's secret codeword.